In [6]:
# First cell - Setup
import sys
from pathlib import Path
import nest_asyncio
import asyncio

# Enable async support
nest_asyncio.apply()

# Add the project root directory to Python path
project_root = str(Path().absolute().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

In [7]:
# Second cell - Fetch data
from providers.nws.api_client import fetch_weather_data

# Test fetching weather data
location = "New York"
frequency = "hourly"
data = fetch_weather_data(location, frequency)

In [8]:
# Second cell - Fetch data
from providers.nws.api_client import fetch_weather_data

# Test fetching weather data
location = "New York"
frequency = "hourly"
data = fetch_weather_data(location, frequency)

In [9]:
# Add this cell after fetching data
print("Raw API Response:")
print("Properties:", data.get('properties'))
if 'properties' in data:
    print("Periods:", len(data['properties'].get('periods', [])))
else:
    print("No properties found in data")
print("---")

Raw API Response:
Properties: {'units': 'us', 'forecastGenerator': 'HourlyForecastGenerator', 'generatedAt': '2024-12-05T23:27:13+00:00', 'updateTime': '2024-12-05T23:24:06+00:00', 'validTimes': '2024-12-05T17:00:00+00:00/P7DT14H', 'elevation': {'unitCode': 'wmoUnit:m', 'value': 4.8768}, 'periods': [{'number': 1, 'name': '', 'startTime': '2024-12-05T18:00:00-05:00', 'endTime': '2024-12-05T19:00:00-05:00', 'isDaytime': False, 'temperature': 34, 'temperatureUnit': 'F', 'temperatureTrend': '', 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 14}, 'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': -6.111111111111111}, 'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 64}, 'windSpeed': '20 mph', 'windDirection': 'W', 'icon': 'https://api.weather.gov/icons/land/night/sct?size=small', 'shortForecast': 'Partly Cloudy', 'detailedForecast': ''}, {'number': 2, 'name': '', 'startTime': '2024-12-05T19:00:00-05:00', 'endTime': '2024-12-05T20:00:00-05:00', 'isDaytime':

In [10]:
# Third cell - Clean and load data
from providers.nws import clean_weather_data, load_weather_data
try:
    print("About to start cleaning...")
    cleaned_data = clean_weather_data(data)
    print(f"Cleaning complete. Got {len(cleaned_data)} records")
    print("Starting database load...")
    asyncio.run(load_weather_data(cleaned_data))
    print(f"Successfully loaded {len(cleaned_data)} weather records")
except Exception as e:
    print(f"Error occurred: {str(e)}")
    import traceback
    traceback.print_exc()

About to start cleaning...
Starting data cleaning...
Found 156 total periods, will process up to 24 future hours

Cleaning complete:
- Processed 24 future forecasts
- Skipped 2 historical periods
Cleaning complete. Got 24 records
Starting database load...
2024-12-05 19:12:52,151 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-05 19:12:52,153 INFO sqlalchemy.engine.Engine INSERT INTO weather_forecasts (location, temperature, relative_humidity, wind_speed, dew_point, is_daytime, probability_of_precipitation, short_forecast, forecast_time, created_at) VALUES ($1::VARCHAR, $2::FLOAT, $3::FLOAT, $4::FLOAT, $5::FLOAT, $6::BOOLEAN, $7::FLOAT, $8::VARCHAR, $9::TIMESTAMP WITH TIME ZONE, now()) RETURNING weather_forecasts.id, weather_forecasts.created_at
2024-12-05 19:12:52,154 INFO sqlalchemy.engine.Engine [cached since 40.95s ago] ('NYC', 33.0, 61.0, 21.0, -6.67, False, 0, 'Partly Cloudy', datetime.datetime(2024, 12, 6, 1, 0, tzinfo=datetime.timezone.utc))
2024-12-05 19:12:52,228 INFO s